In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import uproot
import awkward as ak
import hist
import coffea
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory
import mplhep as hep

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors

import glob
import sys
import os

sys.path.append(os.path.dirname(sys.path[0])) # append project directory to path for import
os.chdir(os.path.dirname(sys.path[0])) # change current working directory to project directory
from processor.schema import JMENanoAODSchema, ScoutingJMENanoAODSchema
from processor.processor import OHProcessor

In [3]:
hep.style.use("CMS")

In [4]:
input_dir = "/eos/user/a/adlintul/QCD_Pt-15To7000_TuneCP5_13p6TeV_pythia8/Run3Summer22"
input_dir = "/data/data/ScoutingJMENano/ScoutingPFMonitor/Run3Summer22"
input_dir = "/data/data/ScoutingJMENano/QCD_Pt-15To7000_TuneCP5_13p6TeV_pythia8/Run3Summer22"
input_dir = "/data/data/JME_NANO_DATA/2022/JMENanoRun3_v2p1_Run2022D-PromptReco-v2/JetMET"
#input_dir = "/data/data/JME_NANO_MC/2022/QCD_Pt-15to7000_TuneCP5_Flat_13p6TeV_pythia8/JMENanoRun3_v2p1_MC22_122"

filelist = glob.glob(os.path.join(input_dir, "*/*/*.root"))

In [5]:
filesize = [os.path.getsize(file) for file in filelist]
maxsize_idx = np.argmax(filesize)
fname = filelist[maxsize_idx]
print("file: {}\nsize: {} MB".format(fname, filesize[maxsize_idx]/1e6))
events = NanoEventsFactory.from_root(
            fname, 
            schemaclass=ScoutingJMENanoAODSchema,
            metadata={"dataset": "test"}
            ).events()

file: /data/data/JME_NANO_DATA/2022/JMENanoRun3_v2p1_Run2022D-PromptReco-v2/JetMET/220915_173253/0000/tree_393.root
size: 447.179081 MB


In [6]:
weight_filelist = ["corrections/JEC/Winter22Run3_V2_MC/Winter22Run3_V2_MC_L1FastJet_AK4PFPuppi.txt",
                   #"corrections/Winter22Run3_V2_MC/Winter22Run3_V2_MC_L2Residual_AK4PFPuppi.txt",
                   "corrections/JEC/Winter22Run3_V2_MC/Winter22Run3_V2_MC_L2Relative_AK4PFPuppi.txt",
                   "corrections/JEC/Winter22Run3_V2_MC/Winter22Run3_V2_MC_L3Absolute_AK4PFPuppi.txt",
                   "corrections/JEC/Winter22Run3_V2_MC/Winter22Run3_V2_MC_L2L3Residual_AK4PFPuppi.txt",
                   "corrections/JEC/Winter22Run3_V2_MC/Winter22Run3_V2_MC_Uncertainty_AK4PFPuppi.junc.txt"]
lumi_json_path = "corrections/lumimask/Cert_Collisions2022_355100_362439_Golden.json"
lumi_csv_path = "corrections/lumidata/lumi2022.csv"

In [7]:
events.Jet.nearest(events.TrigObjJMEAK4)

<PtEtaPhiMCollectionArray [[None, None, None, ... None, None]] type='62544 * var...'>

In [8]:
p = OHProcessor(off_jet_name="Jet", off_jet_label="AK4PUPPI",
                on_jet_name="TrigObjJMEAK4", on_jet_label="AK4HLT",
                #on_jet_name="ScoutingJet", on_jet_label="AK4Scouting",
                save_processed_lumi=True,
                compute_processed_lumi=False,
                lumi_json_path=None, #lumi_json_path,
                lumi_csv_path=None, #lumi_csv_path,
                trigger_min_pt=None,
                off_jet_weight_filelist= None, #weight_filelist,
                on_jet_weight_filelist=None,
                off_rho_name="Rho_fixedGridRhoFastjetAll", 
                on_rho_name="Rho_fixedGridRhoFastjetAll",
                off_jet_tag_probe=False,
                on_jet_tag_probe=False,
                off_jet_tag_min_pt=20,
                on_jet_tag_min_pt=20,
                use_tag_probe = True,
                tag_probe_tag_min_pt = 20,
                is_data=True,
                mix_correction_level=False,
                pt_binning="log",
                eta_binning="fine",
                fill_gen=False,
                gen_jet_name="GenJet",
                gen_jet_label="Gen",
                gen_jet_tag_probe=False,
                gen_jet_tag_min_pt=20,
                ave_jet=False,
                use_weight=False,
                hist_to_fill= "1d" #"tp_response tp_diff_ratio tp_pt_balance tp_mpf"
               )

In [9]:
#%%timeit
out = p.process(events)
#out = p.postprocess(out)
out

{'cutflow': {'test': defaultdict(int,
              {'all events': 62544,
               'lumimask': 62544,
               'NPV > 0': 62535,
               'PV |z| < 24 cm': 62535,
               'PV |r_xy| < 2 cm': 62535,
               'AK4PUPPI >= 1': 62528,
               'AK4HLT >= 1': 5189,
               'trigger PFJetNone (None)': 5189,
               'AK4PUPPI: jet id None': 5189,
               'AK4HLT: jet id None': 5189,
               'AK4PUPPI: jet veto map (off)': 5189,
               'AK4HLT: jet veto map (off)': 5189,
               'AK4PUPPI: JEC (off)': 5189,
               'AK4HLT: JEC (off)': 5189,
               'T&P: Jet and TrigObjJMEAK4': 1371,
               'delta R < 0.2': 870})},
 'processed_lumi': {'test': {'lumi_list': <processor.accumulator.LumiAccumulator at 0x7fadec11a820>}},
 'jet_pt': Hist(
   StrCategory(['test'], growth=True, name='dataset', label='Primary Dataset'),
   StrCategory(['AK4PUPPI_Original', 'AK4HLT_Original'], name='jet_type', label='T

In [10]:
out["tp_response"]

KeyError: 'tp_response'

In [ ]:
# fig, ax = plt.subplots()
# out["pt_response"][{"dataset":"test", "correction_level":"off=Original:on=Original", 
#                     "jet_type":"Gen", "jet_eta":slice(-1.3j, 1.3j, sum), "jet_phi":sum}].plot(ax=ax)
# ax.set_xscale("log")

In [ ]:
# fig, ax = plt.subplots()
# out["jet_pt"]["test", [0, 2, -1], :].plot1d(ax=ax)
# ax.set_xlim(0, 500)
# hep.cms.label(**{"label":"private", "loc":0, "data":False, "year":"2022D", "com":13.6})
# ax.legend()